In [1]:
import sys
sys.path.append('..')
sys.path.append('../ehrshot')
import copy
from typing import Literal
import argparse
import pandas as pd
import numpy as np
import os

import torch
from torch import nn
from torch.distributions import Distribution
from torch_uncertainty.utils.distributions import cat_dist
from torch_uncertainty.routines import ClassificationRoutine
from torch_uncertainty.utils import TUTrainer
from torch_uncertainty.models import deep_ensembles, mc_dropout
from torch_uncertainty.transforms import RepeatTarget
import torchvision.transforms as T

from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
pd.options.display.max_seq_items = 2000

In [2]:
results_dict = {}

labeling_functions=[
    "guo_los",
    "guo_icu",
    "new_hypertension",
    "new_hyperlipidemia",
    "new_acutemi",
    "lab_thrombocytopenia",
    "lab_hyperkalemia",
    "lab_hyponatremia",
    "lab_anemia",
    "lab_hypoglycemia"
]

unique_tasks_1 = ['value_los', 'value_icu']
unique_tasks_2 = ['value_hypoglycemia', 'value_hyperkalemia', 'value_hyponatremia', 'value_anemia', 'value_thrombocytopenia']
unique_tasks_3 = ['value_new_hypertension', 'value_new_hyperlipidemia', 'value_new_acutemi']


all_tasks = [unique_tasks_1, unique_tasks_2, unique_tasks_3]
all_tasks_name = ['general_operation_v1', 'lab_test', 'new_diagnose']

In [3]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(hidden_size, num_classes)
#         self.fc2 = nn.Linear(hidden_size, hidden_size)
#         self.dropout2 = nn.Dropout(p=0.2)
#         self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
#         x = self.dropout2(x)
#         x = F.relu(self.fc3(x))
        return x


def optim_recipe(model, lr_mult: float = 1.0):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01 * lr_mult, momentum=0.9)
    return {"optimizer": optimizer}

# def optim_recipe(model, lr_mult: float = 1.0):
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001 * lr_mult)
#     return {"optimizer": optimizer}

In [4]:
max_epochs = 50
batch_size = 64

# max_epochs = 10
# batch_size = 16

# for i in tqdm(range(len(all_tasks))):
for i in [0]:
    general_task_name = all_tasks_name[i]

    folder_path = f'same_time_data/{general_task_name}'

    train_x_name = os.path.join(folder_path, 'x_train.csv')
    train_y_name = os.path.join(folder_path, 'y_train.csv')
    val_x_name = os.path.join(folder_path, 'x_val.csv')
    val_y_name = os.path.join(folder_path, 'y_val.csv')
    test_x_name = os.path.join(folder_path, 'x_test.csv')
    test_y_name = os.path.join(folder_path, 'y_test.csv')

    X_train = pd.read_csv(train_x_name).to_numpy()
    X_val = pd.read_csv(val_x_name).to_numpy()
    X_test = pd.read_csv(test_x_name).to_numpy()

    for j in tqdm(range(len(all_tasks[i]))):
    # for j in range(1):
        specific_task_name = all_tasks[i][j]
        y_train = pd.read_csv(train_y_name)[specific_task_name].astype(int).to_numpy()
        y_val = pd.read_csv(val_y_name)[specific_task_name].astype(int).to_numpy()
        y_test = pd.read_csv(test_y_name)[specific_task_name].astype(int).to_numpy()

        assert len(np.unique(y_train)) == 2
        # Create class weights
#         class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weights = torch.tensor([1, 1])
        class_weights = torch.tensor(class_weights, dtype=torch.float)

        X_train = torch.tensor(X_train).float()
        X_val = torch.tensor(X_val).float()
        X_test = torch.tensor(X_test).float()

        y_train = torch.tensor(y_train).long()
        y_val = torch.tensor(y_val).long()
        y_test = torch.tensor(y_test).long()

        # Create TensorDatasets
        train_dataset = TensorDataset(X_train, y_train)
        val_dataset = TensorDataset(X_val, y_val)
        test_dataset = TensorDataset(X_test, y_test)

        # Create DataLoaders
        train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        input_size = X_train.shape[1]
        hidden_size = 128
        num_classes = 2
        model = NN(input_size, hidden_size, num_classes)

        trainer = TUTrainer(accelerator="gpu", max_epochs=max_epochs)

        ens_routine = ClassificationRoutine(
            is_ensemble=True,
            num_classes=2,
            model=model,
            loss=nn.CrossEntropyLoss(weight=class_weights),
            format_batch_fn=RepeatTarget(
                1
            ), 
            optim_recipe=optim_recipe(
                model, 1
            ),
            eval_ood=False,
        )

        trainer.fit(ens_routine, train_dataloaders=train_dl, val_dataloaders=val_dl)

        ens_perf = trainer.test(ens_routine, dataloaders=[test_dl])

        results_dict[specific_task_name] = ens_perf
        
        del trainer, ens_routine, model

  0%|                                                     | 0/2 [00:00<?, ?it/s]/tmp/ipykernel_40812/2296140577.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_weights = torch.tensor(class_weights, dtype=torch.float)
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performanc

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/home/zizhang/anaconda3/envs/uq_ehr/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/zizhang/anaconda3/envs/uq_ehr/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/zizhang/anaconda3/envs/uq_ehr/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (38) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/zizhang/anaconda3/envs/uq_ehr/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Test metric  ┃      Classification       ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     Acc      │          95.43%           │
│    Brier     │          0.49462          │
│   Entropy    │          0.01892          │
│     NLL      │          0.53403          │
└──────────────┴───────────────────────────┘
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Test metric  ┃        Calibration        ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     ECE      │          0.04115          │
│     aECE     │          0.03907          │
└──────────────┴───────────────────────────┘
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Test metric  ┃ Selective Classification  ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    AUGRC     │          0.02093          │
│     AURC     │          0.04193          │
│  Cov@5Risk   │          100.00%          │
│  Risk@80Cov  │           4.29%           │
└──────────────┴───────────────────────────┘

100%|█████████████████████████████████████████████| 2/2 [00:22<00:00, 11.31s/it]


In [5]:
# import json
# with open('results_model_uq_v2/results_single_task_baseline.json', 'w') as f:
#     json.dump(results_dict, f)

# import json
# with open('results_model_uq_v3/results_single_task_baseline.json', 'w') as f:
#     json.dump(results_dict, f)

In [6]:
results_dict

{'value_los': [{'test/cal/ECE': 0.2272765189409256,
   'test/cal/aECE': 0.2272760570049286,
   'test/cls/Acc': 0.7079038023948669,
   'test/cls/Brier': 0.5073766324127595,
   'test/cls/NLL': 1.5259760618209839,
   'test/sc/AUGRC': 0.12524858117103577,
   'test/sc/AURC': 0.2343304306268692,
   'test/sc/Cov@5Risk': nan,
   'test/sc/Risk@80Cov': 0.25644171237945557,
   'test/cls/Entropy': 0.15513773262500763,
   'test/ens_Disagreement': nan,
   'test/ens_Entropy': 0.15513773262500763,
   'test/ens_MI': 0.0}],
 'value_icu': [{'test/cal/ECE': 0.04115091264247894,
   'test/cal/aECE': 0.03906511515378952,
   'test/cls/Acc': 0.954344630241394,
   'test/cls/Brier': 0.4946178881388621,
   'test/cls/NLL': 0.5340332388877869,
   'test/sc/AUGRC': 0.0209277905523777,
   'test/sc/AURC': 0.04193313419818878,
   'test/sc/Cov@5Risk': 1.0,
   'test/sc/Risk@80Cov': 0.042944785207509995,
   'test/cls/Entropy': 0.01891803741455078,
   'test/ens_Disagreement': nan,
   'test/ens_Entropy': 0.01891803741455078,